In [ ]:
from langchain_community.chat_models import ChatLlamaCpp
import gc

class ChatLlamaCppManager(ChatLlamaCpp):
  def __init__(self, model_path, **kwargs):
    super().__init__(model_path=model_path, **kwargs)
  


In [41]:
from contextlib import redirect_stderr
from langchain_community.chat_models import ChatLlamaCpp
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.runnables import RunnableConfig

log_path = "/Users/deepesh/Desktop/github-projects/ai-agent/llama_cpp_backend.log"

with open(log_path, "w") as log_file, redirect_stderr(log_file):
    model = ChatLlamaCppManager(
        model_path="/Users/deepesh/llms/mistral-nemo-15.gguf",
        temperature=0.9,
        top_p=0.95,
        n_ctx=2048,
        repeat_penalty=1.2,
        n_gpu_layers=-1,
        verbose=False,
    )

    # Define a new graph
    workflow = StateGraph(state_schema=MessagesState)

    # Define a function that calls model
    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}

    # Define a node in the graph
    workflow.add_edge(START, "model")
    workflow.add_node("model", call_model)

    # Add memory
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    config: RunnableConfig = {"configurable": {"thread_id": "thread_1"}}

    query = "Who was the first president of the United States?"
    input_messages = [HumanMessage(content=query)]

    print("🧠 Response:")
    for chunk, metadata in app.stream({"messages": input_messages}, config=config, stream_mode="messages"):
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="", flush=True)

🧠 Response:
George Washington is considered the first president of the United States. He served two terms in office, from April 30, 1789 to March 4, 1797.

Washington was unanimously elected by the Electoral College for both his terms as President. Before taking up this post, he had played a crucial role during America's Revolutionary War.

In [30]:
from contextlib import redirect_stderr
from langchain_community.chat_models import ChatLlamaCpp
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.runnables import RunnableConfig

log_path = "/Users/deepesh/Desktop/github-projects/ai-agent/llama_cpp_backend.log"

with open(log_path, "w") as log_file, redirect_stderr(log_file):
    model = ChatLlamaCpp(
        model_path="/Users/deepesh/llms/mistral-nemo-15.gguf",
        temperature=0.9,
        top_p=0.95,
        n_ctx=2048,
        repeat_penalty=1.2,
        n_gpu_layers=-1,
        verbose=False,
    )

    # Define a new graph
    workflow = StateGraph(state_schema=MessagesState)

    # Define a function that calls model
    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}

    # Define a node in the graph
    workflow.add_edge(START, "model")
    workflow.add_node("model", call_model)

    # Add memory
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    config: RunnableConfig = {"configurable": {"thread_id": "thread_1"}}

    query = "Who was the first president of the United States?"
    input_messages = [HumanMessage(content=query)]

    print("🧠 Response:")
    for chunk, metadata in app.stream({"messages": input_messages}, config=config, stream_mode="messages"):
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="", flush=True)

🧠 Response:
George Washington is considered the first president of the United States. He served two terms in office, from April 30, 1789 to March 4, 1797.

Washington was unanimously elected by the Electoral College for both his terms as President. Before taking up this post, he had played a crucial role during America's Revolutionary War.